In [4]:
import requests
import pandas as pd

urls = [
    "https://waterservices.usgs.gov/nwis/dv?format=json&siteStatus=all&site=14238000&agencyCd=USGS&statCd=00003&parameterCd=00060&startDT=2024-05-30&endDT=2025-05-29",
    "https://waterservices.usgs.gov/nwis/dv?format=json&siteStatus=all&site=12422500&agencyCd=USGS&statCd=00003&parameterCd=00060&startDT=2024-05-30&endDT=2025-05-29",
    "https://waterservices.usgs.gov/nwis/dv?format=json&siteStatus=all&site=14144700&agencyCd=USGS&statCd=00003&parameterCd=00060&startDT=2024-05-23&endDT=2025-05-22",
    "https://waterservices.usgs.gov/nwis/dv?format=json&siteStatus=all&site=12194000&agencyCd=USGS&statCd=00003&parameterCd=00060&startDT=2024-05-23&endDT=2025-05-22"
]

#list to hold the data frames
dfs = []

#loop through each url
for url in urls:
    response = requests.get(url)
    data = response.json()

    ts = data["value"]["timeSeries"][0]
    site_name = ts["sourceInfo"]["siteName"]
    site_code = ts["sourceInfo"]["siteCode"][0]["value"]
    values = ts["values"][0]["value"]

    df = pd.DataFrame(values)
    df["date"] = pd.to_datetime(df["dateTime"]).dt.date
    df["streamflow_cfs"] = df["value"].astype(int)
    df["site_name"] = site_name
    df["site_code"] = site_code
    df = df[["date", "site_code", "site_name", "streamflow_cfs"]]
    df.set_index("date", inplace=True)

    dfs.append(df)

    print("Loaded site code:")
    print(site_code)
    print("Site name:")
    print(site_name)
    print("Number of rows:")
    print(df.shape[0])
    print("Number of columns:")
    print(df.shape[1])
    print()

combined_df = pd.concat(dfs)
print("Combined DataFrame shape:")
print(combined_df.shape)



Loaded site code:
14238000
Site name:
COWLITZ RIVER BELOW MAYFIELD DAM, WA
Number of rows:
365
Number of columns:
3

Loaded site code:
12422500
Site name:
Spokane River at Spokane, WA
Number of rows:
365
Number of columns:
3

Loaded site code:
14144700
Site name:
COLUMBIA RIVER AT VANCOUVER, WA
Number of rows:
362
Number of columns:
3

Loaded site code:
12194000
Site name:
SKAGIT RIVER NEAR CONCRETE, WA
Number of rows:
365
Number of columns:
3

Combined DataFrame shape:
(1457, 3)


In [5]:
for url in urls:
    response = requests.get(url)
    data = response.json()

    ts = data["value"]["timeSeries"][0]
    site_name = ts["sourceInfo"]["siteName"]
    site_code = ts["sourceInfo"]["siteCode"][0]["value"]
    values = ts["values"][0]["value"]

    df = pd.DataFrame(values)
    df["date"] = pd.to_datetime(df["dateTime"]).dt.date
    df["streamflow_cfs"] = df["value"].astype(int)
    df["site_name"] = site_name
    df["site_code"] = site_code
    df = df[["date", "site_code", "site_name", "streamflow_cfs"]]
    df.set_index("date", inplace=True)

    # --- Hydropower Estimation ---
    Q = df["streamflow_cfs"] * CFS_TO_CMS
    H = HEAD_HEIGHTS[site_code]
    df["power_MW"] = (EFFICIENCY * GRAVITY * Q * H) / 1e6

    dfs.append(df)

    print("DataFrame for site", site_code)
    print(df.head())
    print()


DataFrame for site 14238000
           site_code                             site_name  streamflow_cfs  \
date                                                                         
2024-05-30  14238000  COWLITZ RIVER BELOW MAYFIELD DAM, WA            5260   
2024-05-31  14238000  COWLITZ RIVER BELOW MAYFIELD DAM, WA            5200   
2024-06-01  14238000  COWLITZ RIVER BELOW MAYFIELD DAM, WA            5650   
2024-06-02  14238000  COWLITZ RIVER BELOW MAYFIELD DAM, WA            8340   
2024-06-03  14238000  COWLITZ RIVER BELOW MAYFIELD DAM, WA           10400   

            power_MW  
date                  
2024-05-30  0.035068  
2024-05-31  0.034668  
2024-06-01  0.037668  
2024-06-02  0.055602  
2024-06-03  0.069336  

DataFrame for site 12422500
           site_code                     site_name  streamflow_cfs  power_MW
date                                                                        
2024-05-30  12422500  Spokane River at Spokane, WA            4530  0.020134
2024

In [1]:
EFFICIENCY = 0.8         # 80%
CFS_TO_CMS = 0.0283168   # cubic feet/sec to cubic meters/sec
GRAVITY = 9.81           # m/s²

In [2]:
HEAD_HEIGHTS = {
    "12194000": 112,  # Skagit River (Ross Dam)
    "14238000": 30,   # Columbia River (rough avg)
    "12422500": 20,   # Spokane River
    "14144700": 52    # Cowlitz River (Mayfield Dam)
}

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Step 1: Clean combined DataFrame
combined_df = pd.concat(dfs).reset_index()
combined_df.rename(columns={"date": "Date", "power_MW": "Power (MW)", "site_name": "River"}, inplace=True)
combined_df["Date"] = pd.to_datetime(combined_df["Date"])

# Step 2: Create Figure
fig = go.Figure()

# Add each river as a line
for river in combined_df["River"].unique():
    river_df = combined_df[combined_df["River"] == river]
    fig.add_trace(go.Scatter(
        x=river_df["Date"],
        y=river_df["Power (MW)"],
        mode="lines",
        name=river,
        hovertemplate="%{x|%b %d, %Y}<br>%{y:.2f} MW<extra>%{fullData.name}</extra>"
    ))

# Step 3: Layout Enhancements
fig.update_layout(
    title="Clean Interactive Hydropower Estimate (MW) for WA Rivers",
    xaxis_title="Date",
    yaxis_title="Power Output (MW)",
    template="plotly_white",  # Or try "plotly_dark"
    hovermode="x unified",
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ]
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    legend=dict(title="River", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Step 4: Export to HTML
fig.write_html("clean_hydropower_chart.html", auto_open=True)


